In [13]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

import csv
import requests
import pickle

In [ ]:
%%appyter markdown

# {{ StringField(
    name='title',
    label='Title',
    default='My Title',
).render_value }}

In [ ]:
%%appyter code_exec

# if small molecule name inputted, will go through its CSV file and sort by target affinity

import pandas as pd

sm_name = {{ StringField(
        name = 'Small molecule name',
        label = 'Small molecule name',
        description = 'The small molecule name',
        default = "AC220",
)}}

kinome_df = pd.read_excel('http://lincs.hms.harvard.edu/wordpress/wp-content/uploads/2013/11/HMS-LINCS_KinomeScan_Datasets_2018-01-18.xlsx')
kinome_df['sm_hms_id'] = kinome_df['sm_hms_id'].str.replace(r'\D', '') #remove HMSL before the ID
hms_id = kinome_df.loc[kinome_df['sm_name'] == sm_name, 'sm_hms_id']

# remove the row number
hms_id_list = []
for hms_id in hms_id:
    hms_id_list.append(hms_id)
    
hms_id_string = str(hms_id_list[0])

old_url = 'http://lincs.hms.harvard.edu/db/datasets/20000/results?small+molecules=HMS_ID&output_type=.csv'
new_url = old_url.replace("HMS_ID", hms_id_string) # fills in HMS_ID with the correct small molecule 

data = pd.read_csv(new_url)
df = pd.DataFrame(data)

target_affinity_1 = df.loc[df['Binding Class'] == 1, 'HUGO Gene Symbol']
target_affinity_2 = df.loc[df['Binding Class'] == 2, 'HUGO Gene Symbol']
target_affinity_3 = df.loc[df['Binding Class'] == 3, 'HUGO Gene Symbol']
target_affinity_10 = df.loc[df['Binding Class'] == 10, 'HUGO Gene Symbol'] 

#js = Javascript('alert("Invalid input")')

kinase1_list = []
kinase2_list = []
kinase3_list = []

for kinase in target_affinity_1:
    kinase1_list.append(str(kinase))

if len(kinase1_list) != 0:
    print(sm_name + ' binds to the following kinases with Kd < 100 nM: ')
    kinase1_string = ', '.join(kinase1_list)
    print(kinase1_string)
    print()

for kinase in target_affinity_2:
    kinase2_list.append(str(kinase)) 

if len(kinase2_list) != 0:
    print(sm_name + ' binds to the following kinases with 100 nM ≤ Kd < 1µM: ')
    kinase2_string = ', '.join(kinase2_list)
    print(kinase2_string)
    print()
    
for kinase in target_affinity_3:
    kinase3_list.append(str(kinase))
    
if len(kinase3_list) != 0:
    print(sm_name + " binds to the following kinases with 1µM ≤ Kd < 10 µM: ")
    kinase3_string = ', '.join(kinase3_list)
    print(kinase3_string)

In [ ]:
%%appyter code_exec

# if kinase inputted, return small molecules  

kinase_name = {{ StringField(
        name = 'Kinase name',
        label = 'Kinase name',
        description = 'The kinase name',
        default = "ABL1",
)}}

# retrieve the dictionaries and unpickle
with open('dict_1.pickle', 'rb') as handle:
    dict_1 = pickle.load(handle)
    
with open('dict_2.pickle', 'rb') as handle:
    dict_2 = pickle.load(handle)

with open('dict_3.pickle', 'rb') as handle:
    dict_3 = pickle.load(handle)

# lists for each target affinity for the small molecules
target_aff1 = []
target_aff2 = []
target_aff3 = []

if kinase_name in dict_1:
    target_aff1 = dict_1[kinase_name]
if kinase_name in dict_2:
    target_aff2 = dict_2[kinase_name]
if kinase_name in dict_3:
    target_aff3 = dict_3[kinase_name]

if len(target_aff1) != 0:
    print('Small molecules that bind to ' + kinase_name + ' with Kd < 100 nM: ')
    target1_string = ', '.join(target_aff1)
    print(target1_string)
    print()

if len(target_aff2) != 0:
    print('Small molecules that bind to ' + kinase_name + ' with 100 nM ≤ Kd < 1µM: ')
    target2_string = ', '.join(target_aff2)
    print(target2_string)    
    print()

if len(target_aff3) != 0:
    print('Small molecules that bind to ' + kinase_name + ' with 1µM ≤ Kd < 10 µM: ')
    target3_string = ', '.join(target_aff3)
    print(target3_string)    
    print()